# Stock Market Analysis
##### Shubham Mittal - CMSC320 Final Project Tutorial

### 1. Introduction
Interest in the stock market has been increasing especially after the introduction of more user friendly apps such as Robinhood and Coinbase. Financial data has also started becoming more accessible partly fueling this increase. The pandemic shook the global economy and the market experienced a lot of volatility. As a result, at the beginning of the pandemic, people started moving away from the stock market to more low risk investments. However, as the pandemic has continued, a new surge of people has started coming back to the stock market. 

In this tutorial, I will analyze stock market data in different intervals to identify intraday, weekly, monthly, and yearly trends. These trends could help identify periods of time that may be more lucrative for trading (a certain time of day or a certain month of a year) allowing investors to make better trading decisions. This tutorial will also help you gain a better understanding of the data science pipeline and hopefully allow you to analyze stock market data for your own purposes. 

(Need to add links to sources)
Less likely to invest - https://www.statista.com/topics/7856/covid-19-and-investment-behavior-worldwide/#dossierKeyfigures
Started investing more - https://www.cnbc.com/2021/04/08/a-large-chunk-of-the-retail-investing-crowd-got-their-start-during-the-pandemic-schwab-survey-shows.html


### 2. Required Tools
In order to understand this tutorial and follow along, you will need to have a basic understanding of Python. We will be using Python 3.8 in this tutorial and the following libraries: 
- Pandas
- NumPy
- yfinance 
- yahoofinancials
Pandas and NumPy will be useful to manipulate and store our data. We will be getting financial data from Yahoo Finance using the yfinance and yahoofinancials libraries. 

All of the above libraries can be installed using pip (which is recommended) but the links (NEED TO ADD LINKS TO EACH LIBRARY) provide more detailed information on downloading for specific OS's.